<a href="https://colab.research.google.com/github/sugarforever/LangChain-Tutorials/blob/main/AutoGPT_with_LangChain_Primitives.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain openai tiktoken google-search-results chromadb -qU

In [ ]:
# proxy configuration
from ipywidgets import widgets
import json
import os
import llm_commons.proxy.base

llm_commons.proxy.base.proxy_version = 'aicore'

In [ ]:
from llm_commons.proxy.openai import Completion
from llm_commons.proxy.identity import AICoreProxyClient
from llm_commons.langchain.proxy import ChatOpenAI
from llm_commons.btp_llm.identity import BTPProxyClient
from llm_commons.langchain.proxy import init_llm, init_embedding_model

In [ ]:
with open('config/irpa-d26-genaixl-cx-sec-cn-sk.json') as f:
    sk = json.load(f)

os.environ['AICORE_LLM_RESOURCE_GROUP'] = 'default'
os.environ["TAVILY_API_KEY"] = sk['tavily']

os.environ["BING_SUBSCRIPTION_KEY"] = sk['bing']
os.environ["BING_SEARCH_URL"] = "https://api.bing.microsoft.com/v7.0/search"

os.environ['AICORE_LLM_AUTH_URL'] = sk['url']+"/oauth/token"
os.environ['AICORE_LLM_CLIENT_ID'] = sk['clientid']
os.environ['AICORE_LLM_CLIENT_SECRET'] = sk['clientsecret']
os.environ['AICORE_LLM_API_BASE'] = sk["serviceurls"]["AI_API_URL"]+ "/v2"
os.environ['LLM_COMMONS_PROXY'] = 'aicore'

llm_commons.proxy.resource_group = os.environ['AICORE_LLM_RESOURCE_GROUP']
llm_commons.proxy.api_base = os.environ['AICORE_LLM_API_BASE']
llm_commons.proxy.auth_url = os.environ['AICORE_LLM_AUTH_URL']
llm_commons.proxy.client_id = os.environ['AICORE_LLM_CLIENT_ID']
llm_commons.proxy.client_secret = os.environ['AICORE_LLM_CLIENT_SECRET']

In [ ]:
aic_proxy_client = AICoreProxyClient()
btp_proxy_client = BTPProxyClient()
embedding = init_embedding_model('text-embedding-ada-002', proxy_client=aic_proxy_client, deployment_id='dd88f66091ca1982', api_base=llm_commons.proxy.api_base)
llm = ChatOpenAI(proxy_client=aic_proxy_client, proxy_model_name='gpt-4')

In [ ]:
from langchain_community.utilities import BingSearchAPIWrapper
from langchain.agents import Tool
from langchain.tools.file_management.write import WriteFileTool
from langchain.tools.file_management.read import ReadFileTool

search = BingSearchAPIWrapper()
tools = [
    Tool(
        name = "search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    WriteFileTool(),
    ReadFileTool(),
]


In [ ]:
from langchain.vectorstores import Chroma

vectordb = Chroma(persist_directory="./.chroma", embedding_function=embedding)

In [ ]:
from langchain_experimental.autonomous_agents import AutoGPT
from llm_commons.langchain.proxy import ChatOpenAI

In [ ]:
agent = AutoGPT.from_llm_and_tools(
    ai_name="Xiaoming",
    ai_role="Assistant",
    tools=tools,
    llm=llm,
    memory=vectordb.as_retriever()
)
# Set verbose to be true
agent.chain.verbose = True


In [ ]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as cb:
  agent.run(["点（m,n）关于y=kx+b的对称点坐标是什么"])
  print(cb)

In [ ]:
!cat weather_report.txt